## Webscrape  
* SPARQL https://w.wiki/PgP
* https://kulturpersoner.uppsalakyrkogardar.se/galleri/

In [56]:
#https://kulturpersoner.uppsalakyrkogardar.se/galleri/
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
def cleanupstring(s):
    out_s = s
    while '  ' in out_s:
        out_s = out_s.strip().replace('  ', ' ')
    return out_s, len(s)-len(out_s)

urls = [
    'https://kulturpersoner.uppsalakyrkogardar.se/galleri/']

newList = []

for url in urls:

    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    soup.findAll('a')
    one_a_tag = soup.findAll('a')[0]
    #    print ( one_a_tag )
    #   link = one_a_tag['href']
    time.sleep(1)
    for i in range(1, len(soup.findAll('a'))):
        new_item = dict()
        one_a_tag = soup.findAll('a')[i]
        one_a_tag_href = one_a_tag['href']
        #print(one_a_tag_href)
        #print("\t",one_a_tag.text)
        if   "karta" not in one_a_tag_href \
                and not "Kyrkogårdsvandring" in one_a_tag.text \
                and not "Search" in one_a_tag.text \
                and not "Start" in one_a_tag.text \
                and not "Vandringar" in one_a_tag.text \
                and not "Guidad" in one_a_tag.text \
                and not "galleri" in one_a_tag_href \
                and not "om-oss"  in one_a_tag_href \
                and not "kyrkogardsexpedition"  in one_a_tag_href:
            #print(one_a_tag.text,"|",one_a_tag.text.replace(" ","_").replace("%C3%84","Ä"),"|",one_a_tag_href.replace("/genealogi/",""))
            
            #person = one_a_tag.text.replace("https://kulturpersoner.uppsalakyrkogardar.se/","").replace("-"," ").replace("%C3%84","Ä")
            #person = one_a_tag.text.replace("https://kulturpersoner.uppsalakyrkogardar.se/","")
            new_item["person"] = one_a_tag.text
            new_item["href"] = one_a_tag['href'].replace("https://kulturpersoner.uppsalakyrkogardar.se/","").replace("/","")
            #print(new_item)
            newList.append(new_item)
print (len(newList) ," antal poster")

128  antal poster


In [57]:
import pandas as pd  
Kulturgravar = pd.DataFrame(newList)

In [58]:
Kulturgravar

,person,href
0,Anita Nathorst,anita-nathorst
1,Henri Osti,henri-osti
2,Fadime Sahindal,fadime-sahindal
3,Edvard Raab,edvard-raab
4,Karl Gustaf Lennander,karl-gustaf-lennander
...,...,...
123,Uno von Troil,uno-von-troil
124,Viktor Persson,viktor-persson
125,Vivi Täckholm,vivi-tackholm
126,Yngve Svalander,yngve-svalander


Wikidata 